Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [5]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [12]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [14]:
num_steps = 2001

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 16.369854
Training accuracy: 11.5%
Validation accuracy: 12.8%
Loss at step 100: 3.770354
Training accuracy: 56.0%
Validation accuracy: 56.3%
Loss at step 200: 2.973980
Training accuracy: 65.3%
Validation accuracy: 64.5%
Loss at step 300: 2.639325
Training accuracy: 68.6%
Validation accuracy: 67.5%
Loss at step 400: 2.431976
Training accuracy: 70.6%
Validation accuracy: 69.2%
Loss at step 500: 2.282453
Training accuracy: 71.9%
Validation accuracy: 70.3%
Loss at step 600: 2.166054
Training accuracy: 73.0%
Validation accuracy: 71.0%
Loss at step 700: 2.070972
Training accuracy: 73.7%
Validation accuracy: 71.6%
Loss at step 800: 1.990556
Training accuracy: 74.2%
Validation accuracy: 72.0%
Loss at step 900: 1.920796
Training accuracy: 74.8%
Validation accuracy: 72.3%
Loss at step 1000: 1.859155
Training accuracy: 75.2%
Validation accuracy: 72.6%
Loss at step 1100: 1.803921
Training accuracy: 75.6%
Validation accuracy: 72.8%
Loss at step 1200: 1.753888
Training ac

Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [15]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [16]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 14.590884
Minibatch accuracy: 7.0%
Validation accuracy: 12.5%
Minibatch loss at step 500: 1.338751
Minibatch accuracy: 78.9%
Validation accuracy: 75.1%
Minibatch loss at step 1000: 1.323068
Minibatch accuracy: 77.3%
Validation accuracy: 75.9%
Minibatch loss at step 1500: 0.856433
Minibatch accuracy: 78.1%
Validation accuracy: 76.2%
Minibatch loss at step 2000: 0.980694
Minibatch accuracy: 81.2%
Validation accuracy: 76.4%
Minibatch loss at step 2500: 0.875530
Minibatch accuracy: 78.9%
Validation accuracy: 77.6%
Minibatch loss at step 3000: 1.062211
Minibatch accuracy: 78.1%
Validation accuracy: 78.4%
Test accuracy: 86.6%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [34]:
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(dtype = tf.float32, shape = shape))
def bias_variable(shape):
    return tf.Variable(tf.zeros(shape))

batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    x = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
    y = tf.placeholder(tf.float32, shape=(None, num_labels))
    
    # Variables.
    weights_1 = weight_variable([image_size * image_size, hidden_nodes])
    biases_1 = bias_variable([hidden_nodes])
  
    hidden_inputs = tf.matmul(x, weights_1) + biases_1
    hidden_outputs = tf.nn.relu(hidden_inputs)
    
    weights_2 = weight_variable([hidden_nodes, num_labels])
    biases_2 = bias_variable([num_labels])
    
    logits = tf.matmul(hidden_outputs, weights_2)+biases_2
    
  
    
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

    prediction = tf.nn.softmax(logits)
#     accuracy = 100*tf.reduce_mean(tf.cast(tf.equal(tf.arg_max(prediction, 1), tf.arg_max(y, 1)), dtype = tf.float32))

In [26]:
num_steps = 8001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {x : batch_data, y : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % session.run(accuracy, feed_dict = {x:valid_dataset, y: valid_labels})
    print("Test accuracy: %.1f%%" % accuracy(prediction.eval(feed_dict = {x:test_dataset}), test_labels))

Initialized
Minibatch loss at step 0: 310.072937
Minibatch accuracy: 11.7%
Validation accuracy: 21.3%
Minibatch loss at step 500: 20.757517
Minibatch accuracy: 80.5%
Validation accuracy: 78.2%
Minibatch loss at step 1000: 18.601940
Minibatch accuracy: 80.5%
Validation accuracy: 82.7%
Minibatch loss at step 1500: 6.947844
Minibatch accuracy: 87.5%
Validation accuracy: 82.3%
Minibatch loss at step 2000: 7.032720
Minibatch accuracy: 89.1%
Validation accuracy: 83.4%
Minibatch loss at step 2500: 7.886553
Minibatch accuracy: 87.5%
Validation accuracy: 83.4%
Minibatch loss at step 3000: 10.685275
Minibatch accuracy: 79.7%
Validation accuracy: 84.2%
Minibatch loss at step 3500: 5.197955
Minibatch accuracy: 84.4%
Validation accuracy: 82.0%
Minibatch loss at step 4000: 5.350279
Minibatch accuracy: 86.7%
Validation accuracy: 84.2%
Minibatch loss at step 4500: 3.573992
Minibatch accuracy: 87.5%
Validation accuracy: 84.1%
Minibatch loss at step 5000: 5.761577
Minibatch accuracy: 85.9%
Validation ac

In [37]:
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(dtype = tf.float32, shape = shape))
def bias_variable(shape):
    return tf.Variable(tf.zeros(shape))

batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    x = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
    y = tf.placeholder(tf.float32, shape=(None, num_labels))
    
    # Variables.
    weights_1 = weight_variable([image_size * image_size, hidden_nodes])
    biases_1 = bias_variable([hidden_nodes])
  
    hidden_inputs = tf.matmul(x, weights_1) + biases_1
    hidden_outputs = tf.nn.relu(hidden_inputs)
    
    weights_2 = weight_variable([hidden_nodes, num_labels])
    biases_2 = bias_variable([num_labels])
    
    logits = tf.matmul(hidden_outputs, weights_2)+biases_2
    
  
    
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    prediction = tf.nn.softmax(logits)
    accuracy = 100*tf.reduce_mean(tf.cast(tf.equal(tf.arg_max(prediction, 1), tf.arg_max(y, 1)), dtype = tf.float32))

In [38]:
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {x : batch_data, y : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % session.run(accuracy, feed_dict = {x:batch_data, y: batch_labels}))
            print("Validation accuracy: %.1f%%" % session.run(accuracy, feed_dict = {x:valid_dataset, y: valid_labels}))
    print("Test accuracy: %.1f%%" % session.run(accuracy, feed_dict = {x:test_dataset, y: test_labels}))

Initialized
Minibatch loss at step 0: 401.961212
Minibatch accuracy: 16.4%
Validation accuracy: 13.1%
Minibatch loss at step 500: 18.991232
Minibatch accuracy: 88.3%
Validation accuracy: 78.9%
Minibatch loss at step 1000: 22.371782
Minibatch accuracy: 84.4%
Validation accuracy: 80.3%
Minibatch loss at step 1500: 11.470417
Minibatch accuracy: 86.7%
Validation accuracy: 80.0%
Minibatch loss at step 2000: 8.671576
Minibatch accuracy: 92.2%
Validation accuracy: 81.4%
Minibatch loss at step 2500: 10.361117
Minibatch accuracy: 94.5%
Validation accuracy: 82.2%
Minibatch loss at step 3000: 10.636648
Minibatch accuracy: 87.5%
Validation accuracy: 82.7%
Minibatch loss at step 3500: 9.158730
Minibatch accuracy: 88.3%
Validation accuracy: 82.0%
Minibatch loss at step 4000: 6.718065
Minibatch accuracy: 90.6%
Validation accuracy: 82.5%
Minibatch loss at step 4500: 6.322220
Minibatch accuracy: 92.2%
Validation accuracy: 83.2%
Minibatch loss at step 5000: 5.514723
Minibatch accuracy: 89.1%
Validation 

Validation accuracy: 83.7%
    
Test accuracy: 90.8%